# Superhero (and Supervillain) Name Generator

---

[Superhero Names Dataset](https://github.com/am1tyadav/superhero)

## Task 2

1. Import the data
2. Create a tokenizer
3. Char to index and Index to char dictionaries

In [25]:
!git clone https://github.com/am1tyadav/superhero

fatal: destination path 'superhero' already exists and is not an empty directory.


In [26]:
with open('superhero/superheroes.txt','r') as f:
  data = f.read()

data[:100]

'jumpa\t\ndoctor fate\t\nstarlight\t\nisildur\t\nlasher\t\nvarvara\t\nthe target\t\naxel\t\nbattra\t\nchangeling\t\npyrrh'

In [27]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [28]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [29]:
tokenizer.fit_on_texts(data)

In [30]:
char_to_index = tokenizer.word_index
index_to_char = dict((v,k) for k,v in char_to_index.items())

print(index_to_char)

{1: '\t', 2: 'a', 3: 'e', 4: 'r', 5: 'o', 6: 'n', 7: 'i', 8: ' ', 9: 't', 10: 's', 11: 'l', 12: 'm', 13: 'h', 14: 'd', 15: 'c', 16: 'u', 17: 'g', 18: 'k', 19: 'b', 20: 'p', 21: 'y', 22: 'w', 23: 'f', 24: 'v', 25: 'j', 26: 'z', 27: 'x', 28: 'q'}


## Task 3

1. Converting between names and sequences

In [31]:
names =data.splitlines()
names[:10]

['jumpa\t',
 'doctor fate\t',
 'starlight\t',
 'isildur\t',
 'lasher\t',
 'varvara\t',
 'the target\t',
 'axel\t',
 'battra\t',
 'changeling\t']

In [32]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [33]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [34]:
name_to_seq(names[0])

[25, 16, 12, 20, 2, 1]

In [35]:
def seq_to_name(seq):
  return ''.join([index_to_char[i] for i in seq if i!=0])

In [36]:
seq_to_name([25, 16, 12, 20, 2, 1])

'jumpa\t'

## Task 4

1. Creating sequences
2. Padding all sequences

In [37]:
sequences =[]
for name in names:
  seq = name_to_seq(name)
  if(len(seq)>=2):
    sequences += [seq[:i] for i in range(2, len(seq)+1)]

In [38]:
sequences[:10]

[[25, 16],
 [25, 16, 12],
 [25, 16, 12, 20],
 [25, 16, 12, 20, 2],
 [25, 16, 12, 20, 2, 1],
 [14, 5],
 [14, 5, 15],
 [14, 5, 15, 9],
 [14, 5, 15, 9, 5],
 [14, 5, 15, 9, 5, 4]]

In [39]:
max_len = max([len(x) for x in sequences])
print(max_len)

33


In [40]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences, padding = 'pre',
    maxlen = max_len
)
print(padded_sequences[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 25 16]


In [41]:
padded_sequences.shape

(88279, 33)

## Task 5: Creating Training and Validation Sets

1. Creating training and validation sets

In [42]:
x, y = padded_sequences[:, :-1], padded_sequences[:,-1]
print(x.shape, y.shape)

(88279, 32) (88279,)


In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(66209, 32) (66209,)
(22070, 32) (22070,)


In [44]:
num_chars = len(char_to_index.keys())+1
print(num_chars)

29


## Task 6: Creating the Model

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, LSTM
from tensorflow.keras.layers import Bidirectional, Dense, Dropout
model = Sequential([
                    Embedding(num_chars, 8, input_length = max_len-1),
                    Dropout(0.2),
                    Conv1D(64,5, strides=1, activation='tanh', padding='causal'),
                    MaxPool1D(2),
                    LSTM(32),
                    Dense(num_chars, activation='softmax')

])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 8)             232       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 8)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 64)            2624      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 29)                957       
Total params: 16,229
Trainable params: 16,229
Non-trainable params: 0
__________________________________________________


## Task 7: Training the Model

In [22]:
h = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=50, verbose=2, batch_size=64,
    callbacks=[
               tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
    ]
)

# batch_size=1000
# model.fit(x_train, y_train, batch_size=batch_size, epochs=20, validation_data=(x_test, y_train))


Epoch 1/50
1035/1035 - 39s - loss: 2.8222 - accuracy: 0.1749 - val_loss: 2.6292 - val_accuracy: 0.2131
Epoch 2/50
1035/1035 - 5s - loss: 2.6088 - accuracy: 0.2195 - val_loss: 2.5558 - val_accuracy: 0.2301
Epoch 3/50
1035/1035 - 5s - loss: 2.5615 - accuracy: 0.2309 - val_loss: 2.5161 - val_accuracy: 0.2368
Epoch 4/50
1035/1035 - 5s - loss: 2.5292 - accuracy: 0.2375 - val_loss: 2.4831 - val_accuracy: 0.2409
Epoch 5/50
1035/1035 - 5s - loss: 2.5047 - accuracy: 0.2437 - val_loss: 2.4559 - val_accuracy: 0.2488
Epoch 6/50
1035/1035 - 5s - loss: 2.4845 - accuracy: 0.2488 - val_loss: 2.4380 - val_accuracy: 0.2545
Epoch 7/50
1035/1035 - 5s - loss: 2.4648 - accuracy: 0.2543 - val_loss: 2.4215 - val_accuracy: 0.2589
Epoch 8/50
1035/1035 - 5s - loss: 2.4511 - accuracy: 0.2572 - val_loss: 2.4063 - val_accuracy: 0.2666
Epoch 9/50
1035/1035 - 5s - loss: 2.4389 - accuracy: 0.2610 - val_loss: 2.3918 - val_accuracy: 0.2717
Epoch 10/50
1035/1035 - 5s - loss: 2.4259 - accuracy: 0.2659 - val_loss: 2.3810 -

INFO:tensorflow:Assets written to: /content/superhero/saved_model/assets


INFO:tensorflow:Assets written to: /content/superhero/saved_model/assets


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
!mkdir -p saved_model
model.save('/content/drive/MyDrive/superhero/saved_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/superhero/saved_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/superhero/saved_model/assets


In [51]:
model.save("/content/drive/MyDrive/superhero/trained_model.h5")

## Task 8: Generate Names!

In [23]:
def generate_names(seed):
  for i in range(0,40):
    seq = name_to_seq(seed)
    padded = tf.keras.preprocessing.sequence.pad_sequences([seq], padding='pre', 
                                                           maxlen=max_len-1,
                                                           truncating='pre')
    
    pred = model.predict(padded)[0]
    pred_char = index_to_char[tf.argmax(pred).numpy()]
    seed +=pred_char

    if pred_char == '\t':
      break
  print(seed)

In [24]:
generate_names('rose t')

rose the senter	
